<a href='https://ai.meng.duke.edu'> = <img align="left" style="padding-top:10px;" src=https://storage.googleapis.com/aipi_datasets/Duke-AIPI-Logo.png>

# ResNet Example
In this demo notebook we are going to demonstrate the creation and use of a [ResNet model](https://arxiv.org/pdf/1512.03385.pdf) to see how the ResNet's skip connections can be implemented in code.  The implementations demonstrated here are shallow ResNet model rather than the deep ResNet architectures described in the paper (and available in the [PyTorch model zoo](https://pytorch.org/vision/stable/models.html)).

We will implement ResNet two different ways in this notebook:  
1) Network with no resizing of dimensions in the main path, and pass-through skip connections 
2) Network with resizing of dimensions in the main path, and skip connections using a convolutional layer for resizing to ensure equal size at combination

**Notes:**
- This notebook should be run on GPU

**References:**
- Read the original [ResNet paper](https://arxiv.org/pdf/1512.03385.pdf) for details on the ResNet architecture

In [ ]:
# Run this cell only if working in Colab
# Connects to any needed files from GitHub and Google Drive
import os

# Remove Colab default sample_data
!rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "AIPI540-Deep-Learning-Applications" # Enter repo name
git_path = 'https://github.com/AIPI540/AIPI540-Deep-Learning-Applications.git'
!git clone "{git_path}"

# Install dependencies from requirements.txt file
#!pip install -r "{os.path.join(repo_name,'requirements.txt')}"

# Change working directory to location of notebook
notebook_dir = 'computer_vision/CNNs'
path_to_notebook = os.path.join(repo_name,notebook_dir)
%cd "{path_to_notebook}"
%ls

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torchvision import datasets, transforms
import torchvision
from torch.utils.data import DataLoader, TensorDataset

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchsummary import summary
import urllib.request
import zipfile

TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

## Get data and set up dataloaders

In [ ]:
# Download the data
if not os.path.exists('./data'):
    os.mkdir('./data')
if not os.path.exists('data/hymenoptera_data'):
    url = 'https://download.pytorch.org/tutorial/hymenoptera_data.zip'
    urllib.request.urlretrieve(url,filename='data/hymenoptera_data.zip')
    zip_ref = zipfile.ZipFile('data/hymenoptera_data.zip', 'r')
    zip_ref.extractall('data/')
    zip_ref.close()

In [ ]:
data_dir = 'data/hymenoptera_data'

# Set up transformations for training and validation (test) data
# For training data we will do randomized cropping to get to 224 * 224, randomized horizontal flipping, and normalization
# For test set we will do only center cropping to get to 224 * 224 and normalization
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Create Datasets for training and validation sets
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'),
                                          data_transforms['train'])
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'),
                                          data_transforms['val'])

# Create DataLoaders for training and validation sets
batch_size = 4
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                             shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,
                                             shuffle=False, num_workers=4)

# Set up dict for dataloaders
dataloaders = {'train':train_loader,'val':val_loader}
# Store size of training and validation sets
dataset_sizes = {'train':len(train_dataset),'val':len(val_dataset)}
# Get class names associated with labels
class_names = train_dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
images, labels = iter(train_loader).next()
images = images.numpy()
fig = plt.figure(figsize=(10, 6))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, batch_size//2, idx+1, xticks=[], yticks=[])
    image = images[idx]
    image = image.transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    image = np.clip(image, 0, 1)
    ax.imshow(image)
    ax.set_title("{}".format(class_names[labels[idx]]))

## Version 1: Simple ResNet with passthrough skip connections
Let's first create a ResNet with no resizing of the dimensions in the main path, which means we can use simple skip connections that just pass the block input directly through an identity block without resizing it.

![](img/resnet1.png)

In [ ]:
class ResNetSimple(nn.Module):

    def __init__(self):
        super().__init__()

        ### DOWNSCALING LAYER ###

        # Conv 1 layer: (3,224,224) -> (8,224,224)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1)
        # Conv1 layer output size = (W-F+2P)/S+1 = (224-3+2)/1+1 = 224
        # Batch norm
        self.bn1 = nn.BatchNorm2d(8)
        # Maxpool layer: (8,224,224) -> (8,112,112)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        
        ### RESNET BLOCK 1 ###

        # Conv layer: (8,112,112) -> (8,112,112)
        self.block1conv1 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, stride=1, padding=1)
        # Batch norm
        self.block1bn1 = nn.BatchNorm2d(8)
        # Conv layer: (8,112,112) -> (8,112,112)
        self.block1conv2 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, stride=1, padding=1)
        # Batch norm
        self.block1bn2 = nn.BatchNorm2d(8)

        ### RESNET BLOCK 2 ###

        # Conv layer: (8,112,112) -> (8,112,112)
        self.block2conv1 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, stride=1, padding=1)
        # Batch norm
        self.block2bn1 = nn.BatchNorm2d(8)
        # Conv layer: (8,112,112) -> (8,112,112)
        self.block2conv2 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, stride=1, padding=1)
        # Batch norm
        self.block2bn2 = nn.BatchNorm2d(8)

        ### FINAL LAYERS ###

        # Average pooling layer: (8,112,112) -> (8,16,16)
        self.pool2 = nn.AvgPool2d(kernel_size=7, stride=7)
        
        # Input size: 8 * 16 * 16 = 2048 from pooling layer
        # 2 output channels (for the 2 classes)
        self.fc1 = nn.Linear(8*16*16, 2)
        
    def forward(self, x):

        ### DOWNSCALING LAYER ###
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool1(x)

        ### RESNET BLOCK 1 ###
        skipconnect = x

        x_out = self.block1conv1(x) # conv1
        x_out = self.block1bn1(x_out) # batch norm 1
        x_out = F.relu(x_out) # relu

        x_out = self.block1conv2(x_out) # conv2
        x_out = self.block1bn2(x_out) # batch norm 2

        # Add layer and skipconnect, then activation
        x_out += skipconnect
        x_out = F.relu(x_out)

        ### RESNET BLOCK 2 ###
        skipconnect = x_out

        x_out = self.block2conv1(x_out) # conv1
        x_out = self.block2bn1(x_out) # batch norm 1
        x_out = F.relu(x_out) # relu

        x_out = self.block2conv2(x_out) # conv2
        x_out = self.block2bn2(x_out) # batch norm 2

        # Add layer and skipconnect, then activation
        x_out += skipconnect
        x_out = F.relu(x_out)

        ### FINAL LAYERS ###
        x_out = self.pool2(x_out)
        # Flatten into a vector to feed into linear layer
        x_out = x_out.view(x_out.size(0), -1)
        # Linear layer
        x_out = self.fc1(x_out)
        
        return x_out

In [ ]:
# Instantiate the model
net = ResNetSimple()

# Display a summary of the layers of the model and output shape after each layer
summary(net,(images.shape[1:]),batch_size=batch_size,device="cpu")

### Train the model

In [ ]:
def train_model(model, criterion, optimizer, dataloaders, device, num_epochs=50):

    model = model.to(device) # Send model to GPU if available

    iter_num = {'train':0,'val':0} # Track total number of iterations

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Get the input images and labels, and send to GPU if available
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the weight gradients
                optimizer.zero_grad()

                # Forward pass to get outputs and calculate loss
                # Track gradient only for training data
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backpropagation to get the gradients with respect to each weight
                    # Only if in train
                    if phase == 'train':
                        loss.backward()
                        # Update the weights
                        optimizer.step()

                # Convert loss into a scalar and add it to running_loss
                running_loss += loss.item() * inputs.size(0)
                # Track number of correct predictions
                running_corrects += torch.sum(preds == labels.data)

                # Iterate count of iterations
                iter_num[phase] += 1

            # Calculate and display average loss and accuracy for the epoch
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
    
    return

In [ ]:
# Instantiate the model
net = ResNetSimple()

# Cross entropy loss combines softmax and nn.NLLLoss() in one single class.
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(net.parameters(),  lr=0.001)

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_model(net, criterion, optimizer, dataloaders, device, num_epochs=100)

### Visualize results

In [ ]:
# Display a batch of predictions

def visualize_results(model,dataloader,device):
    model = model.to(device) # Send model to GPU if available
    with torch.no_grad():
        model.eval()
        # Get a batch of validation images
        images, labels = iter(val_loader).next()
        images, labels = images.to(device), labels.to(device)
        # Get predictions
        _,preds = torch.max(model(images), 1)
        preds = np.squeeze(preds.cpu().numpy())
        images = images.cpu().numpy()

    # Plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(15, 10))
    for idx in np.arange(len(preds)):
        ax = fig.add_subplot(2, len(preds)//2, idx+1, xticks=[], yticks=[])
        image = images[idx]
        image = image.transpose((1, 2, 0))
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)
        ax.imshow(image)
        ax.set_title("{} ({})".format(class_names[preds[idx]], class_names[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx] else "red"))
    return

visualize_results(net,val_loader,device)

## Version 2: ResNet with resizing
Let's now create a new ResNet architecture which resizes the input dimensions in the main path (reducing the image dimensions while increasing number of feature maps).  This means we also need to apply a convolution layer in the skip connection branch to resize the data flowing through it, so that its dimensions match the dimensions of the data flowing through the main path in order for them to be combined together.

![](img/resnet2.png)

In [ ]:
class ResNetResize(nn.Module):

    def __init__(self):
        super().__init__()

        ### DOWNSCALING LAYER ###

        # Conv layer: (3,224,224) -> (8,224,224)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1)
        # Conv layer output size = (W-F+2P)/S+1 = (224-3+2)/1+1 = 224
        self.bn1 = nn.BatchNorm2d(8)
        
        ### RESNET BLOCK 1 ###

        # Main path
        # Conv layer: (8,224,224) -> (16,112,112)
        self.block1conv1 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1)
        self.block1bn1 = nn.BatchNorm2d(16)
        # Conv layer: (16,112,112) -> (16,112,112)
        self.block1conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.block1bn2 = nn.BatchNorm2d(16)

        # Skip connection: (8,224,224) -> (16,112,112)
        self.block1skipconv = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=2, padding=1)
        self.block1skipbn = nn.BatchNorm2d(16)

        ### RESNET BLOCK 2 ###

        # Main path
        # Conv layer: (16,112,112) -> (32,56,56)
        self.block2conv1 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.block2bn1 = nn.BatchNorm2d(32)
        # Conv layer: (32,56,56) -> (32,56,56)
        self.block2conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.block2bn2 = nn.BatchNorm2d(32)

        # Skip connection: (16,112,112) -> (32,56,56)
        self.block2skipconv = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.block2skipbn = nn.BatchNorm2d(32)

        ### FINAL LAYERS ###

        # Average pooling layer: (32,56,56) -> (32,8,8)
        self.pool2 = nn.AvgPool2d(kernel_size=7, stride=7)
        
        # Input size: 32 * 8 * 8 =  from pooling layer
        # 2 output channels (for the 2 classes)
        self.fc1 = nn.Linear(32*8*8, 2)
        
    def forward(self, x):

        ### DOWNSCALING LAYER ###
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        ### RESNET BLOCK 1 ###
        skipconnect = x
        skipconnect = self.block1skipconv(skipconnect)
        skipconnect = self.block1skipbn(skipconnect)

        x_out = self.block1conv1(x) # conv1
        x_out = self.block1bn1(x_out) # batch norm 1
        x_out = F.relu(x_out) # relu

        x_out = self.block1conv2(x_out) # conv2
        x_out = self.block1bn2(x_out) # batch norm 2

        # Add layer and skipconnect, then activation
        x_out += skipconnect
        x_out = F.relu(x_out)

        ### RESNET BLOCK 2 ###
        skipconnect = x_out
        skipconnect = self.block2skipconv(skipconnect)
        skipconnect = self.block2skipbn(skipconnect)

        x_out = self.block2conv1(x_out) # conv1
        x_out = self.block2bn1(x_out) # batch norm 1
        x_out = F.relu(x_out) # relu

        x_out = self.block2conv2(x_out) # conv2
        x_out = self.block2bn2(x_out) # batch norm 2

        # Add layer and skipconnect, then activation
        x_out += skipconnect
        x_out = F.relu(x_out)

        ### FINAL LAYERS ###
        x_out = self.pool2(x_out)
        # Flatten into a vector to feed into linear layer
        x_out = x_out.view(x_out.size(0), -1)
        # Linear layer
        x_out = self.fc1(x_out)
        
        return x_out

In [ ]:
# Instantiate the model
net = ResNetResize()

# Cross entropy loss combines softmax and nn.NLLLoss() in one single class.
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(net.parameters(),  lr=0.001)

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_model(net, criterion, optimizer, dataloaders, device, num_epochs=100)